# Read and analyse WhatsApp chat transcripts, and find some insights.
    
    Most chatty person.
    Sentiment analysis for each person.
    Most mentioned.
    High Profanity.
    Topics 

In [46]:
import os
import re

f = '/Users/rammurthys/Downloads'
filename = 'f9_chat.txt'

# members part of this Group chat
members = ['dpk', 'pjk', 'dir', 'srk', 'trk', 'svk', 'krk', 'kk', 'abi','ram','nkk']

phone_book = {'919789987036': 'srk'
              ,'919500566509': 'ram'
              ,'919663655116': 'trk'
              ,'919604854613': 'kk'
              ,'919946628663': 'pjk'
              ,'919840648198': 'dir'
              ,'919677200872': 'svk'
              ,'17164309669': 'dpk'
              ,'919944213734': 'krk'
              ,'918056193719': 'nkk'
              ,'917338813130': 'abi'
             }

# RegEx to parse the text
regexed = re.compile(r'[\W\d\sAMP]+\s+([\w\s]+):(.*)')

In [47]:
# Lets do the clean up
previous = None

content = open(os.path.join(f, filename))

member_content_book = {}

for i in content:
    if 'F9' in i:
        continue
    if regexed.search(i):
        key, val = regexed.search(i).groups()
        member_content_book[key]=member_content_book.get(key, [''])+[val.strip()+' ']
        previous = key
    else:
        member_content_book[previous]=member_content_book.get(previous, [''])+[i.strip()+' ']

In [48]:
# Create pickle file for each member transcript 
# !mkdir transcripts
import pickle

data = {}
for key, val in member_content_book.items():
    data[key]=[''.join(val)]
member_content_book = data

for key, val in member_content_book.items():
    with open('transcripts/'+key+'.txt', 'wb') as fp:
        pickle.dump(val, fp)


In [49]:
# Create a DataFrame
import pandas as pd
pd.set_option('max_colwidth', 150)

df = pd.DataFrame.from_dict(member_content_book).transpose()
df.columns = ['transcript']
df = df.sort_index()

In [50]:
additional_stop_words = [' lam ', ' da ', ' tat ', ' thats ', ' na ', ' adhu ', ' ll ', ' dai ', ' ur ', ' im ', ' u ',
                         ' ithu ', ' ve ', ' la ', ' ah ', ' omitted ', ' ku ', ' oda ', ' dhan ', ' va ', ' erukku ', ' like '
                         ' nu ', ' don ',' nee ' ' unnaku ', ' unaku ', ' enaku ', ' dan ', ' appo ', ' intha ', ' indha '
                         ,' ok ',' okay ']

def cleaning_round1(trans):
    text = re.sub(r"[\d+\W+_]", ' ', trans.lower())
    for i in additional_stop_words:
        text = re.sub(i, ' ', text)
    return text

round1 = lambda x: cleaning_round1(x)

In [51]:
data_clean1 = pd.DataFrame(df.transcript.apply(round1))
data_clean1

,transcript
Deepak Ravishankar Ramkumar,right up your alley macha can convince interviewer years total experience change on relationship status maybe cv jujubi nanba apdi podu g...
Karthik Kottiswaran,watha unga kozha adi sandaila yen thaliya arukura even after so many years you people are still tagging her name with me it s all coz of you ye...
Karthikeyan Narayanaswami,do you really have that much money un per tha varuthu i t have zee account where to watch tamil movies brought by zee not free either ...
Krishna Kumar,unga english padichu meaning kandupudikrathuke time aaguthu oruthan ennada months kanaku bathila week kanaku solran calculate panr...
Raj Kumar,this happens there are companies that t even shortlist your profile if you switched more than companies in your entire profile you re simply n...
Raj Kumar S,u wont be in trouble r the trouble gif i can see ohoh reaction in this place r wasting time thinkinh r guiding ram murthy but the truth is h...
Ram,sticker i saw something like yesterday a jd said years of experience in kubernetes where kubernetes originally introduced less than years ...
Ram Balaji Subbaiyan,image enoda maximum eh yr months in my first job therla dude can add more on this for any visa fillings details a...
Riyas Khan,sticker sticker yaaru master ellah sari aahirum machan sticker compare panrathuku vera yaarum illaya machan ...
Selva Vignesh,wat yenne pannen even i remember this paithiyam thaniya irukule dance class apo po paithiyam aaiduvan i even remember if r staying in cla...


# Organize the data

Corpus development.

Doc-term matrix.

In [52]:
full_names = ['Deepak', 'KarthikK', 'KKeyan', 'Rambi', 'Riyas', 'Selva', 'Krishna', 'Ram', 'RajS', 'RajT']
df

,transcript
Deepak Ravishankar Ramkumar,Right up your alley macha.. u can convince interviewer tat u ve 18 years total experience Change on relationship status maybe .. cv lam jujubi nan...
Karthik Kottiswaran,Watha unga kozha adi sandaila yen thaliya arukura! Even after so many years you people are still tagging her name with me. It’s all coz of you. Ye...
Karthikeyan Narayanaswami,do you really have that much money? 50% un per tha varuthu 😂 I don’t have ZEE5 account. Where to watch tamil movies brought by ZEE5 ? Thats not fr...
Krishna Kumar,@919840648198 unga english lam padichu meaning kandupudikrathuke time aaguthu..oruthan ennada na months kanaku bathila week kanaku solran..adhu ca...
Raj Kumar,This happens. There are companies that don't even shortlist your profile if you've switched more than 3 companies in your entire profile. You're s...
Raj Kumar S,U wont be in trouble U r the trouble ‎GIF omitted I can see ur ohoh reaction in this place 😂😂😂😂 U r wasting ur time thinkinh u r guiding ram murth...
Ram,‎sticker omitted I saw something like yesterday. A JD said 12 years of experience in kubernetes where Kubernetes originally introduced less than 1...
Ram Balaji Subbaiyan,‎image omitted 🤷🏻‍♂ Enoda maximum eh 1 yr 10months in my first Job Therla dude 🤷🏻‍♂ @919663655116 can add more on this 😬 For any VISA fillings . U...
Riyas Khan,‎sticker omitted ‎sticker omitted Yaaru master??? Ellah sari aahirum machan 😂😂😂😂 ‎sticker omitted Compare panrathuku vera yaarum illaya machan🤦🏻‍♂...
Selva Vignesh,Wat 😏😏 Na yenne pannen Even I remember this Paithiyam thaniya irukule dance class la Apo po paithiyam aaiduvan 😂 I don even remember 😂😂😂 If u r st...


In [53]:
# pickle this for later use
df.to_pickle('corpus.pkl')

In [54]:
# Document Term Matrix

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean1.transcript)
data_dtm1 = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm1.index = data_clean1.index
data_dtm1

,aache,aaga,aagura,aaguthu,aahirum,aaide,aaiduvan,aakuna,aal,aalu,...,yevlo,yjmkyefcpl,yosichu,yosikama,youngest,youtu,yr,yrs,zee,zumba
Deepak Ravishankar Ramkumar,1,0,1,0,0,0,0,0,2,1,...,0,0,0,0,0,0,0,0,0,1
Karthik Kottiswaran,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Karthikeyan Narayanaswami,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
Krishna Kumar,0,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
Raj Kumar,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Raj Kumar S,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
Ram,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
Ram Balaji Subbaiyan,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,1,0,0
Riyas Khan,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Selva Vignesh,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [55]:
# pickle Document term matrix for later use
data_dtm1.to_pickle('dtm1.pkl')

# pickle data clean obj, and countvectorizer object too
data_clean1.to_pickle('data_clean1.pkl')
pickle.dump(cv, open('cv.pkl', 'wb'))

In [56]:
d = {}
for key, val in member_content_book.items():
    d[key]= {}
    for i,j in phone_book.items():
        if i in val[0]:
            d[key][j]=d[key].get(j, 0)+1
